In [38]:
# fit ratio
import pandas as pd

datasets = ["linear", "covid", "fb-1", "osm"]
cases = ["10", "21", "22", "41", "42"]
bulkload_ratios = ["0.0", "0.005", "0.01", "0.025", "0.05", "0.1", "0.25", "0.5", "1.0"]
index_types = ["alex", "lipp"]

result = {}

for index_type in index_types:
    fit_counter = 0
    counter = 0
    for dataset in datasets:
        for case in cases:
            for bulkload_ratio in bulkload_ratios:
                if case == "10" and bulkload_ratio != "1.0":
                    continue
                if (case == "21" or case == "22") and bulkload_ratio == "1.0":
                    continue
                if (case == "41" or case == "42") and (bulkload_ratio == "0.0" or bulkload_ratio == "1.0"):
                    continue
                counter += 1;
                file = f"../log/root_smo_profiling/{dataset}/{case}/{bulkload_ratio}/{index_type}_insert_root.log"
                df = pd.read_csv(file)
                if (len(df) == 1):
                    fit_counter += 1
    result[index_type] = (fit_counter, counter)

# print result
print("fit ratio")
for index_type in index_types:
    print(f"{index_type}: {result[index_type][0]}/{result[index_type][1]} = {result[index_type][0]/result[index_type][1]}")
print(f"sum: {result['alex'][0]+result['lipp'][0]}/{result['alex'][1]+result['lipp'][1]} = {(result['alex'][0]+result['lipp'][0])/(result['alex'][1]+result['lipp'][1])}")


fit ratio
alex: 91/124 = 0.7338709677419355
lipp: 100/124 = 0.8064516129032258
sum: 191/248 = 0.7701612903225806


In [45]:
# overfit ratio
import pandas as pd

bulkload_sizes = [0, 1000000, 2000000, 5000000, 10000000, 20000000, 50000000, 100000000]
sizes2ratios = {
    0: "0.0",
    1000000: "0.005",
    2000000: "0.01",
    5000000: "0.025",
    10000000: "0.05",
    20000000: "0.1",
    50000000: "0.25",
    100000000: "0.5",
}
datasets = ["linear", "covid", "fb-1", "osm"]

df = pd.read_csv("./single_thread_thp.csv")
df = df[(df["read_ratio"] != 0)]
df['key_path'] = df['key_path'].apply(lambda x: x.replace('fb', 'fb-1'))
df_grouped = df.groupby(['index_type', 'key_path', 'test_suite', 'init_table_size'])['throughput'].mean().reset_index()
# print(len(df_grouped))

result = {}

for index in ["alex", "lipp"]:
    overfit_counter = 0
    counter = 0
    for dataset in datasets:
        for case in [21, 22, 41, 42]:
            for i in range(len(bulkload_sizes) - 1):
                for j in range(i+1, len(bulkload_sizes)):
                    if (case == 41 or case == 42) and bulkload_sizes[i] == 0:
                        continue
                    # print(f"i:{bulkload_sizes[i]}, j:{bulkload_sizes[j]}")
                    counter += 1
                    # get case throughput
                    d1 = df_grouped[(df_grouped["key_path"] == f"datasets/{dataset}") & (df_grouped["init_table_size"] == bulkload_sizes[i]) & (df_grouped["test_suite"] == case) & (df_grouped["index_type"] == index)]["throughput"].values
                    d2 = df_grouped[(df_grouped["key_path"] == f"datasets/{dataset}") & (df_grouped["init_table_size"] == bulkload_sizes[j]) & (df_grouped["test_suite"] == case) & (df_grouped["index_type"] == index)]["throughput"].values
                    root_model_file1 = f"../log/root_smo_profiling/{dataset}/{case}/{sizes2ratios[bulkload_sizes[i]]}/{index}_insert_root.log"
                    root_model_file2 = f"../log/root_smo_profiling/{dataset}/{case}/{sizes2ratios[bulkload_sizes[j]]}/{index}_insert_root.log"
                    p1 = pd.read_csv(root_model_file1)
                    p2 = pd.read_csv(root_model_file2)
                    is_fit = True if len(p1) == 1 and len(p2) == 1 else False
                    if is_fit == True and d1[0] > d2[0] and d1[0] / d2[0] > 1.2:
                        overfit_counter += 1
                        # print(f"index: {index}, case:{case}, dataset:{dataset}, bulkload_size: {bulkload_sizes[i]}/{bulkload_sizes[j]}, {d1}/{d2}={d1/d2}, model_change: {is_model_change}")
    result[index] = (overfit_counter, counter)

# print result
print("overfit (throughput change ratio > 1.2) ratio")
for index in ["alex", "lipp"]:
    print(f"{index}: {result[index][0]}/{result[index][1]} = {result[index][0]/result[index][1]}")
print(f"sum: {result['alex'][0]+result['lipp'][0]}/{result['alex'][1]+result['lipp'][1]} = {(result['alex'][0]+result['lipp'][0])/(result['alex'][1]+result['lipp'][1])}")


overfit (throughput change ratio > 1.2) ratio
alex: 33/392 = 0.08418367346938775
lipp: 5/392 = 0.012755102040816327
sum: 38/784 = 0.04846938775510204
